In [1]:
from sklearn import datasets, model_selection
import xgboost as xgb

In [2]:
import numpy as np
import pandas as pd

In [3]:
X_train = pd.read_csv('./UltimateFinal/Regression/X_train.csv', index_col=0)
y_train = pd.read_csv('./UltimateFinal/Regression/y_train.csv', index_col=0)
X_test = pd.read_csv('./UltimateFinal/Regression/X_test.csv', index_col=0)
y_test = pd.read_csv('./UltimateFinal/Regression/y_test.csv', index_col=0)

In [4]:
X = X_train.append(X_test)
X = X.iloc[:,2:]

/var/folders/kn/5p9n18rd2r58j5lzlvq6zh040000gn/T/ipykernel_20841/3720978659.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X_train.append(X_test)


In [5]:
y = y_train.append(y_test)

/var/folders/kn/5p9n18rd2r58j5lzlvq6zh040000gn/T/ipykernel_20841/3621025881.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y = y_train.append(y_test)


In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.08, random_state=1)

In [7]:
X_train

,Arts & Photography,Biographies & Memoirs,Business & Money,Childrens Books,Christian Books & Bibles,Comics & Graphic Novels,Computers & Technology,Cookbooks,Crafts,Education & Teaching,...,Category_Politics & Social Sciences,Category_Reference,Category_Religion & Spirituality,Category_Romance,Category_Science & Math,Category_Science Fiction & Fantasy,Category_SelfHelp,Category_Sports & Outdoors,Category_Teen & Young Adult,Category_Travel
96856,0.000000,0.000000,5.000000,3.833333,1.000000,3.0,0.0,3.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
65822,0.000000,4.000000,3.000000,0.000000,5.000000,0.0,0.0,5.000000,4.0,4.0,...,0,0,0,0,0,0,0,0,0,0
243306,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4146,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
259772,4.857143,4.333333,4.333333,4.500000,4.666667,0.0,0.0,5.000000,5.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371403,5.000000,4.750000,0.000000,5.000000,0.000000,0.0,0.0,5.000000,5.0,5.0,...,0,0,0,0,0,0,0,0,0,0
491263,0.000000,4.000000,4.000000,0.000000,0.000000,0.0,0.0,4.882353,5.0,0.0,...,0,0,0,0,0,0,0,0,0,0
470924,3.000000,3.320000,3.500000,0.000000,0.000000,0.0,0.0,4.000000,0.0,3.0,...,0,0,0,0,0,0,0,0,0,0
491755,5.000000,4.666667,4.750000,5.000000,0.000000,0.0,0.0,3.000000,3.5,0.0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
y_train

,Rating
96856,3
65822,5
243306,5
4146,3
259772,4
...,...
371403,5
491263,4
470924,3
491755,3


In [9]:
regressor=xgb.XGBRegressor(eval_metric='rmse')

In [62]:
#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
from sklearn.model_selection import GridSearchCV
# set up our search grid
param_grid = {"max_depth":    [5],
              "n_estimators": [10],
              "learning_rate": [0.1]}

# try out every combination of the above values
search = GridSearchCV(regressor, param_grid, cv=2, verbose=3).fit(X_train.iloc[:,2:], y_train)

print("The best hyperparameters are ",search.best_params_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END learning_rate=0.1, max_depth=5, n_estimators=10;, score=-1.628 total time=   1.6s
[CV 2/2] END learning_rate=0.1, max_depth=5, n_estimators=10;, score=-1.622 total time=   1.6s
The best hyperparameters are  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 10}


In [63]:
regressor=xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],)

regressor.fit(X_train.iloc[:,2:], y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=5, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [64]:
train_predictions = regressor.predict(X_train.iloc[:,2:])
test_predictions = regressor.predict(X_test.iloc[:,2:])

In [66]:
train_predictions

array([3.3389592, 3.3171892, 2.3876095, ..., 3.0040052, 2.3226748,
       3.32824  ], dtype=float32)

In [71]:
from sklearn.metrics import mean_squared_error
is_MSE = mean_squared_error(y_train, train_predictions)
print("The MSE is %.5f" % is_MSE )


from sklearn.metrics import mean_absolute_error
is_MAE = mean_absolute_error(y_train, train_predictions)
print("The MAE is %.5f" % is_MAE )


num = sum((train_predictions - y_train.iloc[:,0])**2)
den = sum((np.mean(y_train.iloc[:,0]) - y_train.iloc[:,0])**2)
is_R2 = 1 - (num/den)
print("The R2 is %.5f" % is_R2)

The MSE is 2.38938
The MAE is 1.40021
The R2 is -1.62357


In [72]:
oos_MSE = mean_squared_error(y_test, test_predictions)
print("The MSE is %.5f" % oos_MSE )


oos_MAE = mean_absolute_error(y_test, test_predictions)
print("The MAE is %.5f" % oos_MAE )


num = sum((test_predictions - y_test.iloc[:,0])**2)
den = sum((np.mean(y_train.iloc[:,0]) - y_test.iloc[:,0])**2)
oos_R2 = 1 - (num/den)
print("The R2 is %.5f" % oos_R2)

The MSE is 2.38889
The MAE is 1.39912
The R2 is -1.60540


In [73]:
X_train["XGB_predictions"] = train_predictions
X_test["XGB_predictions"] = test_predictions

In [75]:
X = X_train.append(X_test)

/var/folders/kn/5p9n18rd2r58j5lzlvq6zh040000gn/T/ipykernel_32387/560222870.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X_train.append(X_test)


In [ ]:
books = pd.read_csv('./UltimateFinal/Books.csv')

In [88]:
top_books_9512 = X[X["User"] == 9512].sort_values(by='XGB_predictions', ascending=False).iloc[:10,1]
top_books_53 = X[X["User"] == 53].sort_values(by='XGB_predictions', ascending=False).iloc[:10,1]

In [98]:
top_books_9512_info = books[books["Item"].isin(top_books_9512)]
top_books_53_info = books[books["Item"].isin(top_books_53)]

In [101]:
top_books_9512_info.to_csv('./UltimateFinal/Recommendations/9512/xgboost.csv', index=False)
top_books_53_info.to_csv('./UltimateFinal/Recommendations/53/xgboost.csv', index=False)